# 15장 - 제품으로 배포하기

In [ ]:
# 딥러닝 모델 추론을 대규모로 수행하도록 인프라 구조를 관리하는 것은 비용이나 아키텍처 측면에서 인상적임
# 파이토치는 다양한 양산용 기능이 추가되면서 대규모 제품화까지 지원하며 전 분야를 아우르는 엔드투엔드 플랫폼으로 발전함

In [ ]:
# 제품으로 배포한다는 이야기는 사용 사례에 따라 다음과 같이 해석 가능함
# 1. 모델에 접근 가능한 네트워크 서비스 설정하기: 플라스크(Flask)와 새니(Sanic) 두가지 경량 파이썬 웹 프레임워크 사용
# 2. 모델을 표준화된 포맷으로 내보내어 최적화된 모델 프로세서나 특수 하드웨어, 클라우드 서비스에 출시할 수 있도록 준비. 파이토치 모델은 이를 위해 ONNX(open neural network exchange) 사용
# 3. 더 큰 애플리케이션의 일부로 통합하고 싶을 경우: 모델이 파이썬으로 제한되지 않아야 좋으므로 다른 언어로의 징검다리 역할하는 C++파이토치 모델 사용
# 4. 모바일 기기에서 동작: 최근 파이토치는 모바일 지원하기 시작함

In [ ]:
# 14장의 분류기 사용해 서빙한 다음 얼룩말 모델 사용해 배포 방식 설명

## 1절 - 파이토치 모델 서빙

In [ ]:
# 모델 서버에 올리는 것부터 시작
# 서버 만들고 단순 동작을 확인하고 단점 확인한 후 개선
# 최종 완성물 살펴보고 네트워크 요청을 대기하도록 제작

#### .1 플라스크에 들어간 모델

In [ ]:
# 플라스크는 파이썬 모듈 중 가장 많이 사용되며 pip로 설치 가능
# API는 데코레이터 사용하여 만듦
# flask_hello_world.py
from flask import Flask
app = Flask(__name__)

@app.route("/hello")
def hello():
    return "Hello World!"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

In [ ]:
# 실행하면 애플리케이션은 8000번 포트 듣고 있으며, /hello 경로로 들어오는 요청에 대해 Hello World 문자열 리턴하도록 동작
# 이 플라스크 서버에 앞서 저장했던 모델을 로딩하여 POST 경로로 연결 가능

In [ ]:
# 데이터는 플라스크의 request를 통해 데이터 입력받음
# 정확히는 request.files는 필드 이름으로 접근할 수 있는 파일 객체의 딕셔너리 포함
# 입력은 JSON으로 파싱하고 플라스크의 jsonify 헬퍼를 사용하여 JSON 문자열로 반환

In [ ]:
# /hello 대신 /predict 경로 열어 바이너리 블록(시리즈 데이터의 픽셀 내용)과 연관된 메타데이터(키가 shape인 딕셔너리 포함하는 JSON 객체)를 POST 요청으로 제공된 입력 파일로 받고 예측한 결과를 JSON응답으로 리턴
# 데이터를 얻기 위해 가장 먼저 해야 할 작업은 JSON을 디코드해 바이너리로 바꾸는 것이고, 바꾼 바이너리는 numpy.frombuffer를 사용해 1차원 배열로 디코딩
# 이 배열을 torch.from_numpy를 사용해 텐서와 실제 차원 정보에 맞는 뷰로 변환
# 모델링은 LunaModel을 인스턴스로 만들고 훈련에서 얻은 가중치를 로딩한 후 모델을 eval 모드로 설정
# 훈련 필요 없으므로 with torch.no_grad()블록에서 모델 돌릴 때 기울기 필요없다고 알려줌

In [ ]:
# flask_server.py
import numpy as np
import sys
import os
import torch
from flask import Flask, request, jsonify
import json

from p2ch13.model_cls import LunaModel

app = Flask(__name__)

model = LunaModel() # 모델 설정하고 가중치 읽어들 후 평가 모드로 전환
model.load_state_dict(torch.load(sys.argv[1],
                                 map_location='cpu')['model_state'])
model.eval()

def run_inference(in_tensor):
    with torch.no_grad(): # 자동미분 없음
        # LunaModel takes a batch and outputs a tuple (scores, probs)
        out_tensor = model(in_tensor.unsqueeze(0))[1].squeeze(0)
    probs = out_tensor.tolist()
    out = {'prob_malignant': probs[1]}
    return out

@app.route("/predict", methods=["POST"]) # /predict 엔드포인트에서 폼 입력(HTTP_POST) 받음
def predict():
    meta = json.load(request.files['meta']) # request는 meta로 부르는 파일 하나 가짐
    blob = request.files['blob'].read()
    in_tensor = torch.from_numpy(np.frombuffer(
        blob, dtype=np.float32)) # 바이너리 블롭을 토치로 변환
    in_tensor = in_tensor.view(*meta['shape'])
    out = run_inference(in_tensor)
    return jsonify(out) # 응답 콘텐츠를 JSON으로 인코딩

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)
    print (sys.argv[1])

In [ ]:
# 서버 띄우기
python3 -m p3ch15.flask_server data/part2/models/cls_2019-10-19_15.48.24_final_cls.best.state

In [ ]:
# 샘플 하나 보내는 간단한 클라이언트 cls_client.py 구현함
# 디렉토리에서 실행
python p3ch15/cls_client.py

In [ ]:
# 실행 시 해당 결절은 악성 아닐 가능성이 높다고 출력됨
# 서버가 입력을 받아 모델을 실행하여 결과를 출력

#### .2 배포 시 바라는 점

In [ ]:
# 모델 서빙 시 고려할 점
# 최신 프로토콜과 기능 지원해야 함
# 부분적인 개선 만들어내기 위해 새닉으로 프레임워크를 업그레이드하면 효율적으로 계산 가능
# 특히 GPU 사용하는 경우 배치로 여러 요청을 모아 처리하는 것이 효율적임
# 여러 커넥션에서 복수의 요청을 모은 작업을 배치로 묶어 GPU에서 실행한 후 결과를 각 요청에 리턴
# 하나의 GPU에 여러 개의 배치 돌릴 이유는 없으므로 배치 크기를 최대로 올리는 것이 일반적으로 더 효율적임

In [ ]:
# 여러 작업을 병렬로 수행하길 원함
# 비동기 서빙의 두 번째 스레드에서 동작하는 모델도 효율적으로 동작하게 만들길 원함
# 즉 파이썬 GIL로부터 모델이 벗어나게 만들고 싶음

In [ ]:
# 메모리 복사도 최소로 줄이고 싶음
# 메모리 사용량과 시간 측면에서 뭔가를 계속 복사하는 방식은 나쁨
# HTTP에서는 Base64 인코딩을 많이 사용함
# 부분적인 개선을 만들기 위해 스트리밍 PUT 요청으로 Base64 문자열을 할당하지 않도록 만들고 문자열을 뒤에 덧붙여 가며 늘리는 일 피하기

In [ ]:
# 안전성 문제
# 자원 고갈이나 오버플로 방지를 위해 안전한 디코딩 방식 원함
# 고정된 크기의 입력으로 시작한 파이토치는 충돌 일으키기 어렵지만 이를 위해 이미지 디코딩하고 크기 조정하는 식의 방법은 골 아프고 보장하기 어려움
# 책에서는 패스

#### .3 배치 요청

In [ ]:
# 두 번째 서버는 새닉 프레임워크를 사용
# 새닉을 이용하면 비동기 처리 방식으로 여러 요청을 병렬로 서빙 가능
# 요청을 배치로 만드는 것도 구현

In [ ]:
# 배치 요청 처리하려면 요청 처리 과정에서 모델 실행 분리해야 함

In [ ]:
# 구현
# 두 개의 함수로 구현하기

In [ ]:
# 모델 실행 함수는 한번 시작하면 영원히 동작
# 모델을 실행할 필요 있을 때마다, 모델 실행 함수는 입력을 배치로 구성하고 두 번째 스레드에서 모델을 실행한 다음 결과 반환

In [ ]:
# 요청 프로세서는 요청을 디코딩하고 입력을 큐에 넣은 후 처리가 끝나기를 기다린 다음, 출력 결과를 담아 반환
# 새 요청이 들어오면 큐에 넣고 필요하면 처리하도록 시그널 준 후 요청에 대한 응답으로 보내기 전에 결과를 기다림
# 배치가 요청으로 꽉 차거나 제일 먼저 들어온 요청의 대기 시간이 타이머를 넘으면 이벤트 처리 시작

In [ ]:
# 대부분의 주요 코드는 ModelRunner 클래스에서 확인 가능
# request_batching_server.py:32
class ModelRunner:
    def __init__(self, model_name):
        self.model_name = model_name
        self.queue = []

        self.queue_lock = None # lock으로 동작

        self.model = get_pretrained_model(self.model_name,
                                          map_location=device) # 모델 읽어 인스턴스화 시킴. JIT으로 전환 시 이 부분만 수정하면 됨. 현재는 p3ch15/cyclegan.py에서 사이클 GAN 읽어들임

        self.needs_processing = None # 모델 실행 시그널

        self.needs_processing_timer = None # 타이머

In [ ]:
# ModelRuuner는 먼저 모델 읽어들인 후 몇 가지 관리 작업 수행
# 모델 외에 다른 구성 요소도 필요

In [ ]:
# queue 수정 시 다른 작업이 큐 바꾸지 못하도록 asynio 모듈에서 제공하는 asynio.Lock인 queue_lock 사용
# 여기서 사용하는 모든 asynio 객체는 이벤트 루프 알아야 하며 이 루프는 애플리케이션 초기화가 끝난 후에 사용 가능
# 따라서 초기화 작업 때에는 임시로 None 설정
# 워커가 여러 개인 경우 락 지켜볼 필요 있음
# 주의사항: 파이썬의 비동기 락은 스레드 안전(threadsafe)하지 않음

In [ ]:
# 할 일 없는 경우 ModelRuuner는 대기
# 대기 마치고 일하는 시점을 시그널로 알리는 것은 RequestProcessor의 몫
# 실제로는 asynio.Event인 needs_processing을 통해 진행
# ModelRuuner는 needs_process 이벤트를 기다리기 위해 wait() 메소드 사용
# RequestProcessor는 시그널 주기 위해 set() 사용하며 ModelRunner가 깨어나면 clear()로 이벤트 시그널 리셋함

In [ ]:
# 최대 대기 시간 보앟기 위해 타이머 설정
# 필요할 때 app.loop.call_at으로 만들 수 있으며, 타이머는 needs_processing 이벤트를 예비로 하나 만들어 둠
# 타이머 다 되기 전 배치 처리하는 경우 만들어진 타이머용 이벤트 클리어하여 불필요한 호출 방지

In [ ]:
# 요청부터 큐까지
# 다음으로 요청을 큐에 넣기
# 첫 번째 async 메소드인 process_input에서 수행
# request_batching_server.py:54
async def process_input(self, input):
    our_task = {"done_event": asyncio.Event(loop=app.loop), # task 데이터 설정
                "input": input,
                "time": app.loop.time()}
    async with self.queue_lock: # lock 건 후 태스크 추가
        if len(self.queue) >= MAX_QUEUE_SIZE:
            raise HandlingError("I'm too busy", code=503)
        self.queue.append(our_task)
        logger.debug("enqueued task. new queue size {}".format(len(self.queue)))
        self.schedule_processing_if_needed() # 스케줄링. 프로세싱 메소드는 배치가 다 찼으면 needs_processing 켜기. 배치가 다 차지 않았고 타이머 설정되지 않았다면 최대 대기 시간에 깨어 날수 있도록 타이머 켜기

    await our_task["done_event"].wait() # 처리 끝날 때까지 기다리기. 실행은 await로 루프에 넘기기
    return our_task["output"]

In [ ]:
# 태스크 정보 보관을 위해 파이썬 딕셔너리를 사용
# 딕셔너리에는 input, 큐에 들어간 시간인 time, 작업 처리 후에 켜질 done_event 들어 있음
# 프로세싱은 여기에 output추가

In [ ]:
# 큐의 락을 유지하면서, 태스크를 큐에 넣고 필요에 따라 프로세싱을 스케줄링 함
# 큐 가득 차 있다면 에러 발생시킴
# 이후 태스크 끝나길 기다리고 반환만 하면 됨

In [ ]:
# 큐에서 가져와 배치 실행하기
# request_batching_server.py:71
async def model_runner(self):
    self.queue_lock = asyncio.Lock(loop=app.loop)
    self.needs_processing = asyncio.Event(loop=app.loop)
    logger.info("started model runner for {}".format(self.model_name))
    while True:
        await self.needs_processing.wait() # 처리할 작업 들어올 때까지 대기
        self.needs_processing.clear()
        if self.needs_processing_timer is not None: # 타이머 설정되어 있으면 클리어
            self.needs_processing_timer.cancel()
            self.needs_processing_timer = None
        async with self.queue_lock:
            if self.queue:
                longest_wait = app.loop.time() - self.queue[0]["time"]
            else:  # oops
                longest_wait = None
            logger.debug("launching processing. queue size: {}. longest wait: {}".format(len(self.queue), longest_wait))
            to_process = self.queue[:MAX_BATCH_SIZE] # 배치 얻어, 필요한 경우 다음 배치 실행을 스케줄링
            del self.queue[:len(to_process)]
            self.schedule_processing_if_needed()
        # so here we copy, it would be neater to avoid this
        batch = torch.stack([t["input"] for t in to_process], dim=0)
        # we could delete inputs here...

        result = await app.loop.run_in_executor(
            None, functools.partial(self.run_model, batch) # 별도 스레드에서 모델 연산 수행하고 데이터를 디바이스로 옮겨 모델로 전달. 끝난 후 프로세싱의 나머지 수행
        )
        for t, r in zip(to_process, result): # 결과를 작업 아이템에 더하고 이를 알리는 이벤트 켜기
            t["output"] = r
            t["done_event"].set()
        del to_process

In [ ]:
# model_ruuner는 설정 후 무한 루프 돌음
# 앱 시작할 때 호출되므로 queue_lock과 needs_processing 이벤트를 설정할 수 있음
# 이후 루프 안에서 needs_processing 이벤트를 await시킴

In [ ]:
# 이벤트 들어왔을 때 타이머가 설정됐다면 어차피 지금 프로세스 진행할 예정이므로 테이머 리셋
# model_runner는 큐에서 배치를 꺼내서 필요하면 다음번 배치 프로세싱을 스케줄링 함
# 이때 개별 태스크에서 배치를 만들고 모델 평가를 위한 스레드를 asynio의 app.loop.run_in_executor를 사용해 만듦
# 마지막으로 출력을 태스크에 더하고 done_event를 켬

In [ ]:
# asynio와 await가 뿌려진 플라스크 같은 웹 프레임워크는 래퍼가 필요
# model_runner 함수를 이벤트 루프에서 시작해야 함
# 여러 스레드가 큐에서 꺼내오면서 서로를 인터럽트하는 상황 아니면 큐에 락은 필요 없음
# 다른 프로젝트에 코드 적용할 때를 대비해 요청을 누락하는 가능성 없애도록 락을 사용

In [ ]:
# 서버 시작
python -m p3ch15.request_batching_server data/p1ch2/horse2zebra_0.4.0.pth

In [ ]:
# data/p1ch2/horse.jpg 업로드하여 테스트 후 결과 저장
curl -T data/p1ch2/horse.jpg http://localhost:8000/image --output /tmp/res.jpg

In [ ]:
# 이 서버는 GPU를 위해 요청을 배치로 만들고 비동기로 동작하는 등 잘 만들어졌지만 파이썬 모드를 사용하므로 메인 스레드에서 요청을 처리할 때 GIL로 인해 병렬 처리하기가 어려우며 인터넷 같은 적대적인 환경에서는 위험에 노출되기 쉬움
# 요청 데이터를 디코딩하는 부분은 성능이 최적화되어 있지도 않고 안전하지도 않음

## 2절 - 모델 내보내기

In [ ]:
# GIL이 개선된 웹서버 방해할 가능성이 있어 파이썬 인터프리터에서 파이토치 사용하는 방식이 늘 좋지는 않음
# 혹은 파이썬 돌리기에 성능 문제가 있거나 불가능한 임베디드 시스템 사용해야 될 수도 있음
# 해결책
# 파이토치를 들어내고 전용 프레임워크로 옮겨가기
# 혹은 파이썬의 파이토치 서브셋을 위한 JIT(just in time) 컴파일러 사용하며 파이토치 환경에 머무를 수 있음
# 파이썬 안에서 JIT 사용해도 꽤 괜찮은 최적화 해주고 GIL에서 벗어나게 도와줌
# 시간이 좀 걸리지만 모델을 파이토치가 제공하는 C++ 라이브러리인 libtorch나 여기서 파생된 토치 모바일(torch mobile)로 돌리는 방법

#### .1 ONNX로 파이토치를 넘어서는 호환성 확보

In [ ]:
# 경우에 따라서 모델이 파이썬 생태계 밖에서 동작해야 할 경우 있음
# 임베디드 하드웨어에서 동작하기 위한 특별한 모델 배포 파이프라인 사용
# 이를 위해 ONNX(open neural network exchange)라는 신경망과 머신러닝 모델을 위한 상호 호환 포맷 존재
# 이 포맷으로 내보내면 모델은 ONNX 런타임과 같은 동일한 ONNX 호환 런타임 환경을 사용하는 모든 곳에서 실행 가능
# 모델이 사용하는 연산이 ONNX 표준과 타깃 런타임에서 지원하는 경우만 해당
# 라즈베리 파이에서 파이토치 직접 실행하는 것보다 이 방식이 더 빠른 경우 존재
# 일반적인 하드웨어 외에 꽤 많은 특수 AI 가속 하드웨어에서 ONNX 지원

In [ ]:
# 딥러닝 모델은 행렬곱, 컨볼루션, relu, tanh 같은 매우 특별한 명령어 집합으로 이뤄진 프로그램
# 계산을 직렬화할 수 있다면 저수준 명령어를 이해하는 다른 런타임 환경에서 실행 가능
# ONNX는 이런 연산과 파라미터를 기술하는 표준화된 포맷

In [ ]:
# 오늘날 대부분의 딥러닝 프레임워크는 사용하는 계산을 ONNX로 직렬화할 수 있도록 지원
# ONNX 파일 읽어 실행도 가능(파이토치는 불가능)
# 엣지(edge) 디바이스라고 하는 저사양 단말 장치 중에는 ONNX 파일을 입력으로 받아 특정 디바이스를 위한 저수준 명령을 만들기도 함
# 최근의 클라우드 컴퓨팅 제공사들은 ONNX 파일을 업로드하면 REST 엔드포인트를 통해 서비스를 공개하게 만들기도 함

In [ ]:
# 모델을 ONNX로 내보내려면 모델을 빈 입력으로 실행해야 함
# 입력 텐서의 값은 상관 없고 차원 정보와 타입만 맞으면 됨
# torch.onnx.export 함수를 호출해 파이토치는 모델이 수행한 계산을 추적하고 주어진 파일명으로 ONNX 파일에 직렬화해서 기록함
torch.onnx.export(seg_model, dummy_input, "seg_model.onnx")

NameError: name 'torch' is not defined

In [ ]:
# 결과로 만들어진 ONNX 파일은 런타임에서 실행 가능하고 엣지 디바이스용으로 컴파일 될 수 있으며 클라우드 서비스에도 업로드 됨
# 파이썬에서 onnxruntime이나 onnxruntime-gpu를 설치 후 batch를 넘파이 배열로 얻어 사용 가능
# onnx_example.py (깃허브에 없음)
import onnxruntime

sess = onnxruntime.InferenceSession("seg_model.onnx") # ONNX 런타임 API는 모델의 정의하기 위해 세션을 사용하며 키 값 현태의 입력값들을 사용하여 run 메소드를 호출. 정적 그래프에 정의된 연산을 다루기 위한 전형적인 설정으로 생각하면 됨
input_name = sess.get_inputs()[0].name
pred_onxx, = sess.run(None, {input_name: batch})

In [ ]:
# 모든 토치스크립트 연산을 표준화된 ONNX 연산으로 나타낼 수 있는 것은 암
# 특별한 연산을 ONNX로 내보내면 런타임에 알 수 없는 aten 연산이라는 에러 발생

#### .2 파이토치로 내보내기: 추적

In [ ]:
# 호환성은 중요하지 않지만 파이썬 GIL은 피하고 싶거나 다른 이유로 신경망 모델을 내보내야 하는 경우, 토치스크립트 그래프라는 파이토치만의 표현 방식 사용 가능

In [ ]:
# 토치스크립트 만드는 제일 단순한 방법은 추적(tracing)하는 것인데 ONNX 내보내기와 거의 비슷함
# ONNX 모델 내부에서 일어나는 동작과 같음
# torch.jit.tract 함수를 사용해 모델에 빈 입력 넣고, UNetWrapper 읽어들이고, 훈련시킨 파라미터 볼러온 후 모델 평가 모드로 만들기

In [ ]:
# 모델 추적 전 주의할 점으로 파라미터 중 어떤 것도 기울기가 필요하면 안됨
# torch.no_grad() 콘텍스트 관리자는 엄격히 런타임 스위치이기 때문
# 모델을 no_grad 안에서 추적하도록 만들고 밖에서 실행해도 파이토치는 기울기를 기록
# 모델 따라 내려가고 나서 파이토치에 실행을 요청
# 추적된 모델은 저장된 연산 실행할 때 기울기가 필요한 파라미터가 있게 되어 기울기 필요한 상황이 유발되므로, 추적된 모델을 torch.no_grad 콘텍스트 안에서 실행

In [ ]:
# torch.jit.trace 호출
# trace_example.py (깃허브에 없음)
import torch
from p2ch13.model_seg import UNetWrapper
seg_dict = torch.load('...')
seg_model = UNetWrapper(...)
seg_model.load_state_dict(seg_dict['model_state'])
seg_model.eval()
for p in seg_model.parameters(): # 기울기 요청하지 않도록 파라미터 설정
    p.requires_grad_(False)

dummy_input = torch.randn(1, 8, 512, 512)
traced_seg_model = torch.jit.trace(seg_model, dummy_input) # 추적

In [ ]:
# 추적 기능은 경고를 출력
# 유넷에서 크롭하기 때문에 발생하는 메시지
# 모델에 512*512 이미지만 넣을 것이면 문제 안됨

In [ ]:
# 추적된 모델은 다음과 같이 저장 가능
torch.jit.save(traced_seg_model, 'traced_seg_model.pt')

In [ ]:
# 이후 저장한 파일 있으면 읽어들인 후 호출 가능
loaded_model = torch.jit.load('traced_seg_model.pt')
prediction = loaded_model(batch)

In [ ]:
# 파이토치 JIT가 모델 정장할 때 평가 모드로 동작하지 않고 파라미터도 기울기 값 사용하지 않는다는 설정 보존
# 저장할 때 이 부분 설정하지 않았다면 실행할 때 with torch.no_grad() 사용해야 함

#### .3 추적된 모델로 만든 서버

In [ ]:
# 웹서버 최종 버전 제작
# 추적된 사이클GAN 모델을 내보내기
python3 p3ch15/cyclegan.py data/p1ch2/horse2zebra_0.4.0.pth
-> data/p3ch15/traced_zebra_model.pt

In [ ]:
# 서버에서는 get_pretrained_model 호출을 torch.jit.lead로 바꾸기만 하면 됨
# 이제부터 바라던 대로 모델이 GIL로부터 독립해서 동작
# 편의를 위해 request_batching_jit_server.py를 살짝 수정
# 추적된 모델 파일 경로는 명령행 인자로 받을 수 있음

## 3절 - 파이토치 JIT 동작

In [ ]:
# 파이토치 JIT는 풍부한 배포 옵션을 제공하는 것과 더불어, 파이토치에서 일어난 최근의 여러 가지 핵심적인 혁신 중 하나

#### .1 전통적인 파이썬/파이토치를 넘어서기

In [ ]:
# 파이썬에서 모델 돌리지 않아 얻는 속도 면에서의 장점은 멀티스레드 환경에서만 나타나는 것 중요
# 중간 과정은 파이썬 객체가 아니기 때문에 파이썬 병렬화 이슈인 GIL이 연산에 영향 주지 않음

In [ ]:
# 다음 연산 보기 전에 현재 연산 하나를 수행하는 식의 전통적인 파이토치 방식에서 벗어나면 연산에 대한 전체적인 관점 가질 수 있음
# 계산을 전체적인 관점에서 생각하는 것
# 이를 통해 고도화된 최적화나 높은 수준의 변환으로 가는 문 열 수 있음
# 보통 추론(inference) 시점에 고려하지만 훈련에서도 최적화를 통해 상당한 성능 개선을 가져오기도 함

In [ ]:
# 예제를 통해 여러 연산 한꺼번에 고려하는 것의 이득 확인
# 파이토치가 GPU 상에서 일련의 연산들을 수행할 때 각각에 대해 서브 프로그램(CUDA 용어로 커널)을 호출
# 모든 커널은 GPU 메모리에서 입력을 얻어온 후 결과를 계산하고 저장
# 대부분의 시간을 메모리에 읽고 쓰는 작업에 사용
# 따라서 한번에 읽어 여러 연산 수행하고 마지막에 모두 쓰면 성능이 개선됨
# 이 아이디어는 파이토치 JIT fuser가 하는 작업 반영함

In [ ]:
# 5개의 입력과 2개의 출력과 7개의 중간 결과물 있는 LSTM 빌딩 블록에서 일어나는 단위 계산
# 모두를 하나의 CUDA 함수로 연산하고 중간 결과물을 레지스터에 두면 JIT의 메모리 읽기 횟수 12->5, 쓰기 연산 9->2, 신경망 훈련 시간 4배 감소

In [ ]:
# 파이썬을 벗어나기 위해 JIT를 사용한 속도 개선은 파이썬이 느리다는 소문에 비해서 대단하지 않지만, GIL을 회피하는 부분은 멀티스레드 애플리케이션에서 중요
# JIT 모델에서 대부분의 성능 개선은 JIT에서 제공하는 특수한 최적화 덕분이며 파이썬 오버헤드 피하는 것보다 더 정교한 최적화 과정 포함

#### .2 인터페이스와 백엔드 관점에서의 파이토치

In [ ]:
# 파이썬을 사용하지 않고 동작하는지 이해하기 위해 파이토치를 여러 부분으로 나눠 생각
# torch.nn 모듈은 신경마의 파라미터를 보관하며 함수형 인터페이스를 사용해 텐서 입력받고 텐서 출력
# 이들은 C++ 확장으로 구현되었고 C++ 수준의 자동미분이 활성화된 계층으로 넘겨짐

In [ ]:
# C++ 함수가 이미 존재하므로 파이토치 개발자들은 이를 공식 API로 만듦
# 이것이 LibTorch의 핵심이며, 대응하는 파이썬의 텐서 연산을 우리가 직접 C++로 구현할 수 있게 해줌
# torch.nn 모듈은 파이썬에서만 사용할 수 있지만 C++ API가 이들을 각각 torch::nn 네임스페이스에 대응하여 파이썬처럼 보이지만 파이썬 언어에서 독립하여 사용할 수 있도록 설계됨

In [ ]:
# 따라서 파이썬에서 했던 작업을 동일하게 C++에서 재현 가능하지만 목표는 모델을 내보내는 것임
# 파이토치에는 동일한 함수 제공하는 다른 인터페이스 JIT 존재
# 파이토치 JIT은 연산에 대한 심볼릭 표현 제공
# 표현이라 함은 토치스크립트 혹은 토치스크립트IR을 의미
# 토치스크립트 모듈 읽고 검사하며 실행하기 위해 다룬 파이토치 API나 파이토치 JIT 함수들은 파이썬과 C++ 둘 다에서 접근 가능

In [ ]:
# C++와 파이썬 각각에서 함수 직접 호출하거나 중개자로서 JIT를 거쳐 호출
# 모든 경우는 결국 C++ LibTorch함수 호출하고 ATen과 연산 백엔드로 내려감

#### .3 토치스크립트

In [ ]:
# 토치스크립트 모델 만드는 방법에는 추적에 의한 것과 스크립트에 의한 것 두 가지 방법
# 최상위 수준에서 두 가지 방법의 동작 방식 확인

In [ ]:
# 추적
# 위에서 사용했던 추적의 경우 파이토치 모델을 샘플 입력을 사용해서 실행했음
# 파이토치 JIT는 허용하는 모든 함수에 대한 연산 레코딩을 위한 후크(hook) 있음
# JIT는 파이토치 함수가 호출돼야 동작하며 추적 중에는 모든 파이썬 코드 실행 가능하지만 JIT는 파이토치만 인지함
# 일반적으로 정수 튜플로 이뤄진 텐서의 차원 정보를 사용하면 JIT 동작을 확인하려 시도하지만 중간에 포기할 수 있음(유넷 추적 때 경고 떴던 이유)

In [ ]:
# 스크립트
# 파이토치 JIT는 연산을 위한 실제 파이썬 코드를 보고 토치스크립트 IP로 컴파일 함
# 즉 작성한 프로그램을 JIT가 모두 캡처하는 과정은 컴파일러가 이해할 수 있는 범위로 제한된다는 의미

In [ ]:
# 첫 번째 차원에 비효율적인 덧셈을 수행하는 함수로 추적과 스크립트 수행
import torch

def myfn(x):
    y = x[0]
    for i in range(1, x.size(0)):
        y = y + x[i]
    return y

In [ ]:
# 위 함수 추적하기
inp = torch.randn(5,5)
traced_fn = torch.jit.trace(myfn, inp)
print(traced_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  y0 = torch.add(y, torch.select(x, 0, 1))
  y1 = torch.add(y0, torch.select(x, 0, 2))
  y2 = torch.add(y1, torch.select(x, 0, 3))
  return torch.add(y2, torch.select(x, 0, 4))



In [ ]:
# 코드가 다섯 개의 행으로 인덱싱과 덧셈이 고정되어 4개 행이나 6개 행인 경우에 의도대로 동작하지 않음
# 이런 때 다음과 같은 스크립트 필요
scripted_fn = torch.jit.script(myfn)
print(scripted_fn.code)

def myfn(x: Tensor) -> Tensor:
  y = torch.select(x, 0, 0)
  _0 = torch.__range_length(1, torch.size(x, 0), 1)
  y0 = y
  for _1 in range(_0):
    i = torch.__derive_index(_1, 1, 1)
    y1 = torch.add(y0, torch.select(x, 0, i))
    y0 = y1
  return y0



In [ ]:
# 토치스크립트 내부 표현에 가까운 스크립트 그래프도 출력 가능
xprint(scripted_fn.graph)
# end::cell_5_code[]

# tag::cell_5_code[]

NameError: name 'xprint' is not defined

In [ ]:
# 실제로는 torch.jit.script를 테코레이터 형태로 쓰게 되는 일이 빈번할 것임
@torch.jit.script
def myfn(x):
    ...

In [ ]:
# 입력값 중의하며 커스텀 trace 데코레이터를 만들어 쓸 수도 있음 (책에서 하지 않음)

In [ ]:
# 토치스크립트가 파이썬의 서브셋 같아 보일 수 있지만 근본적 차이 존재
# 파이토치는 코드 안에 타입 명세 추가 -> 정적 타이핑
# 프로그램의 모든 변수 값이 하나의 타입만 가지고 이 타입은 토치스크립트IP 표현으로 제한됨
# 프로그램 안에서 JIT는 통상 자동으로 타입을 추론하지만 스크립트 함수에서 텐서가 아닌 인자의 경우 타입은 애노테이션으로 명세해야 함

In [ ]:
# 이제 추적이나 스크립트 모델을 사용할 수 있음
# 이 모델들은 우리가 즐겨 쓰는 모듈과 거의 비슷하게 동작
# 추적과 스크립트 모두 샘플 입력과 함께 torch.jit.trace나 torch.jit.script로 Module 인스턴스를 전달
# 각각은 우리가 사용하던 forward 메소드 넘겨줌
# 스크립트 경우에 외부에서 호출 가능한 다른 메소드를 공개하고 싶다면 클래스 정의에서 @torch.jit.export로 데코레이션하면 됨

In [ ]:
# JIT로 처리된 모듈이 파이썬처럼 동작한다는 사실은 훈련에도 적용됨
# JIT 모듈을 사용하는 훈련은 일반적인 모델처럼 인터페이스를 설정할 필요가 있다는 의미

In [ ]:
# 사이클GAN, 분류 모델, 유넷 세그멘테이션과 같은 상대적으로 단순한 알고리즘 모델의 경우에는 모델을 추적할 수 있음
# 복잡한 모델에서 쓰기 좋은 점을 들면, 다른 스크립트로 만든 코드나 추적 코드를 스크립트로 된 함수나 추적되는 함수에서 사용할 수 있고, 모듈을 만들어 추적하거나 스크립트로 작성할 때에도  스크리브로 되었거나 추적되는 서브 모듈 사용할 수 있다는 점
# 또한, nn.Module 호출로 함수를 추적할 수 있는데 이때에는 기울기 요구하지 않도록 파라미터를 설정하여 추적되는 모델에 대해 파라미터가 상수로 취급되도록 만듦

#### .4 추적 가능하도록 토치스크립트로 만들기

In [ ]:
# 자연어 처리에서 쓰이는 순환 신경망이나 탐지를 위한 빠른 R-CNN 패밀리 같은 복잡한 모델에서는 for 루프를 위한 제어 흐름을 스크립트로 작성해야 함
# 이때 유연성을 원했다면 추적 중 경고 메시지를 출력한 코드를 발견하게 될 것임
# util/unet.py
class UNetUpBlock(nn.Module):
    def center_crop(self, layer, target_size):
        _, _, layer_height, layer_width = layer.size()
        diff_y = (layer_height - target_size[0]) // 2
        diff_x = (layer_width - target_size[1]) // 2
        return layer[:, :, diff_y:(diff_y + target_size[0]), diff_x:(diff_x + target_size[1])] # 추적 중 경고 메시지 출력하는 부분

In [ ]:
# 여기서 JIT는 차원 정보를 가진 튜플 up.shape의 정보를 유지한 채 1차원 정수 텐서로 바꿈
# 이제 [2:]나 diff_x와 diff_y는 모두 추적 가능한 텐서 연산
# 하지만 이렇게 바뀌어도 슬라이싱은 파이썬 int가 필요한데 여기서 JIT가 끝나므로 경고 메시지 보여줌

In [ ]:
# center_crop 스크립트 작성해서 문제 해결
# 호출하는 지점과 호출되는 지점 사이에서 center_crop 스크립트에 up을 전달하고 이 스크립트에서 크기를 추출함
# 그러고 @torch.jit.script 데코레이터 추가하면 됨
# 코드는 깃허브에 없음

In [ ]:
# 스크립트화할 수 없는 것들 모두 C++로 구현한 커스텀 연산으로 옮겨넣는 등의 방법도 있음
# 토치비전 라이브러리는 마스크 R-CNN 모델 내부 특수 연산에 대해 커스텀 연산으로 옮겨 넣는 방법 사용

## 4절 - LibTorch: C++ 파이토치

In [ ]:
# C++로 직접 모델 내보내는 법

In [ ]:
# 말-얼룩말 사이클GAN JIT 모델을 C++ 프로그램으로 돌리기

#### .1 JIT로 처리된 모델을 C++로 실행하기

In [ ]:
# CImg 라이브러리 사용

In [ ]:
# JIT로 처리된 모델 실행
# cyclegan_jit.cpp
#include "torch/script.h" # 파이토치 스크립트 헤더와 네이티브 JPEG 지원 CImge 인클루드
#define cimg_use_jpeg
#include "CImg.h"
using namespace cimg_library;
int main(int argc, char **argv) {
// end::part1[]
  if (argc != 4) {
    std::cerr << "Call as " << argv[0] << " model.pt input.jpg output.jpg"
              << std::endl;
    return 1;
  }
  CImg<float> image(argv[2]); # 이미지 읽어 디코딩하여 float 배열에 삽입
  # 입력으로부터 여기에서 출력 텐서 생성해야 함
  CImg<float> out_img(output.data_ptr<float>(), output.size(2), # data_prt<float>()메소드는 텐서 저장소에 해당하는 포인터 반환. 이 포인터와 차원정보를 사용해 출력 이미지 생성
                      output.size(3), 1, output.size(1));
  out_img.save(argv[3]); # 이미지 저장
  return 0;
}

In [ ]:
# 파이토치를 위해 C++ 헤더 torch/script.h 인클루드
# 이후 CImge 라이브러리 인클루드하고 설정
# main 함수에서 명령행 인자로 주어진 파일의 이미지 읽고 CImge로 리사이즈해서 227*227 이미지 가지는 CImge<float> 변수 image 얻음
# 프로그램 끝에는 (1, 3, 277, 277) 텐서에서 동일한 타입의 out_img 만들어 저장

In [ ]:
# 이미지에서 입력 텐서 만들고 모델 로딩해 입력 텐서로 실행하는 실제 연산 부분
#cyclegan_jit.cpp
auto input_ = torch::tensor(
  torch::ArrayRef<float>(image.data(), image.size())); # 이미지 데이터 텐서에 입력
  auto input = input_.reshape({1, 3, image.height(),
			       image.width()}).div_(255); # 리세이프 후 크기 조정을 해 CImge 형태에서 파이토치 형으로 변경

  auto module = torch::jit::load(argv[1]); # JIT로 처리된 모델이나 함수를 파일에서 읽어들이기

  std::vector<torch::jit::IValue> inputs; # 입력을 원소 하나인 IValue 벡터로 패킹
  inputs.push_back(input);
  auto output_ = module.forward(inputs).toTensor(); # 모듈을 호출하고 결과 텐서를 추출. 효율성 높이기 위해 소유권이 이동되었기에, IValue 유지하면 나중에 비어 있게 됨

  auto output = output_.contiguous().mul_(255); # 결과를 메모리에서 연속적인 형태로 만듦

SyntaxError: invalid syntax (<ipython-input-46-555e16bac6b9>, line 3)

In [ ]:
# 파이토치가 특정 순서로 다량의 메모리 공간에서 값을 기록하고 있음
# CImge도 마찬가지로 image.data()로 이 메모리에 대한 float 배열 포인터를 얻을 수 있고 image.size()로 요소 개수를 얻을 수 있음
# 이 둘로 더 똑똑한 레퍼런스인 torch:ArrayRef를 만들 수 있고 이것으로 torch::tensor 생성자를 사용해 리스트처럼 파싱할 수 있음

In [ ]:
# 현재 텐서는 1차원이므로 리세이프 해야 함
# CImge는 파이토치와 같은 순서(채널, 행, 열) 사용
# CImge는 0...255 범위 사용하고 우리 모델은 0...1 사용하므로 나눴다가 나중에 다시 곱할 것임

In [ ]:
# 추적할 모델 읽는 것은 torch::jit::load를 사용하면 간단하게 할 수 있음
# 파이썬과 C++ 사이 연결하는 역할 하는 파이토치 추상화에 대해 다루기
# 입력은 모든 값에 대한 제네릭 타입인 IValue로 래핑해야 함
# JIT 내부 함수로 IValue의 벡터가 전달되므로 이렇게 선언하고 입력 텐서에 push_back 해야 함
# 이러면 텐서를 자동적으로 IValue로 래핑하게 됨
# 순방향으로 IValue의 벡터를 넣어주면 한 개의 IValue를 돌려받음
# 결과 IValue에서 .toTensor를 사용해 다시 텐서를 얻을 수 있음

In [ ]:
# 이를 통해 IValue에는 타입이 있다는 점(여기서는 Tensor)과 복수의 int64_t나 double, 텐서 리스트 등을 가질 수 있다는 점 확인 가능
# 여러 출력이 있는 경우 텐서 리스트 가지는 IValue 얻는데, 파이썬 호출 컨벤션으로 만들어졌을 것임
# .toTensor로 IValue에 있는 텐서 꺼내면 IValue는 소유권 변경해줌(즉 무효화)
# 경우에 따라 모델이 인접한 데이터 반환하지 않을 수 있음
# CImge에는 연속적인 블럭 제공해야 하므로 contiguous 호출
# 주어진 메모리 안에서 작업할 때 연속적인 텐서를 스코프 내의 변수로 할당하는 부분은 중요
# 파이썬처럼 파이토치는 더 이상 사용하지 않는 텐서의 메모리는 해제함

In [ ]:
# 파이토치 링킹하는 작업은 복잡하므로 견본 CMake 파일 사용
# CMakeLists.txt
cmake_minimum_required(VERSION 3.0 FATAL_ERROR)
project(cyclegan-jit) # 프로젝트 이름

find_package(Torch REQUIRED) # 토치를 필요한 패키지로 설정
set(CMAKE_CXX_FLAGS "${CMAKE_CXX_FLAGS} ${TORCH_CXX_FLAGS}")

add_executable(cyclegan-jit cyclegan_jit.cpp) # cyclegan_jit.cpp 소스 파일 컴파일 후 cyclegan-jit 실행파일 생성
target_link_libraries(cyclegan-jit pthread jpeg X11) # CImg에 필요한 라이브러리 링킹. CImg 자체는 인클루드만으로 사용 가능하기에 등장하지 않음
target_link_libraries(cyclegan-jit "${TORCH_LIBRARIES}")
set_property(TARGET cyclegan-jit PROPERTY CXX_STANDARD 14)

In [ ]:
# 소스 코드 디렉토리 아래 빌드 디렉토리 만드는 방법이 제일 좋음
# CMake를 실행 후 마지막으로 make 실행
# 빌드 끝나면 cyclegan-git 프로그램 만들어지며 실행 가능

In [ ]:
# 파이썬 없이 파이토치 모델 실행 완료
# 애플리케이션 출시를 원하면 라이브러리를 실행 파일이 있는 곳으로 복사해서 항상 찾을 수 있게 만들어야 함

#### .2 시작부터 C++: C++ API

In [ ]:
# C++ 모듈러 API는 파이썬처럼 보이도록 만들어짐
# 사이클GAN 생성기를 JIT 없이 C++만으로 정의된 버전으로 변경
# 사전 훈련된 가중치는 필요하므로 모델을 추적한 버전을 저장

In [ ]:
# 코드 관리에 필요한 세부 내용인 인클루드와 네임스페이스부터 시작
#include <torch/torch.h>  # torch/torch.h 헤더와 CImg 인클루드
#define cimg_use_jpeg
#include <CImg.h>
using torch::Tensor;  # torch::Tensor를 main 네임스페이스에 추가

SyntaxError: invalid syntax (<ipython-input-54-8d13c1ad7632>, line 5)

In [ ]:
# ConvTransposed2d가 임의로 정의된 것 확인 가능
# C++ 모듈러 API는 개발 중이고 파이토치 1.4에서 만들어진 ConvTransposed2d 모듈은 두 번째 인자로 옵션값 받기 때문에 Sequential에서 사용 불가
# 2장의 파이썬 CycleGAN 생성기와 같은 구조 가지게 만들 것임

In [ ]:
# 잔차 블럭 확인
# cyclegan_cpp_api.cpp
struct ResNetBlock : torch::nn::Module {
  torch::nn::Sequential conv_block;
  ResNetBlock(int64_t dim)
      : conv_block(  # Sequential 초기화 후 서브모듈 추가
            torch::nn::ReflectionPad2d(1),
            torch::nn::Conv2d(torch::nn::Conv2dOptions(dim, dim, 3)),
            torch::nn::InstanceNorm2d(
	       torch::nn::InstanceNorm2dOptions(dim)),
            torch::nn::ReLU(/*inplace=*/true),
	    torch::nn::ReflectionPad2d(1),
            torch::nn::Conv2d(torch::nn::Conv2dOptions(dim, dim, 3)),
            torch::nn::InstanceNorm2d(
	       torch::nn::InstanceNorm2dOptions(dim))) {
    register_module("conv_block", conv_block); # 할당할 모듈 등록하지 않으면 문제 발생
  }

  Tensor forward(const Tensor &inp) {
    return inp + conv_block->forward(inp); # 예상했던 대로 순방향 전파는 단순함
  }
};

In [ ]:
# 파이썬에서처럼 torch::nn::Module의 서브 클래스 등록
# 잔차 블럭은 연속된 conv_block 서브 모듈 가짐

In [ ]:
# 마찬가지로 파이썬에서처럼 서브 모듈을 초기화해야 함
# 이를 위해 C++ 초기화 문 사용
# 파이썬의 __init__ 생성자에서 서브 모듈을 구성했던 것과 유사
# 파이썬에서 멤버로 할당하고 등록하기 위해 __setattr__ 리다이렉트를 지원하기 위한 인트로스펙션이나 후킹 같은 기능 C++에서 제공 안 함

In [ ]:
# 키워드 인자가 없기 때문에 기본 인자로 파라미터를 정의하면 어색하므로 모듈은 통상 options 인자 받음
# 파이썬 옵션 키워드 인자는 옵션 객체의 메소드에 대응하고 여러 개를 이어서 작성 가능

In [ ]:
# 멤버 등록과 할당은 항상 동기화되도록 주의해야 함
# 하지 않으면 예상치 못한 일 발생

In [ ]:
# 파이썬과 대조적으로 모듈에 대해 m->forward(...)를 호출할 필요도 있음
# 어떤 모듈은 직접 가능하지만 Sequential은 안됨

In [ ]:
# 호출 규약에 대해, 함수에 제공한 텐서를 어떻게 수정하느냐에 따라 텐서에 대한 텐서 인자를 수정하지 않은 경우에는 const Tensor&로, 수정한 경우에는 Tensor로 항상 전달해야 함
# 텐서는 Tensor로 반환해야 함
# (Tensor&)처럼 상수가 아닌 레퍼런스 형태의 인자 타입을 반환하면 컴파일러 과정 중 파싱 에러 유발함

In [ ]:
# 주 생성기 클래스에서는 클래스 이름을 전형적인 C++ API 형태로 사용하고, TORCH_MODULE 매크로 사용해 토치 모듈인 ResNetGenerator로 승격할 것임
# 모듈을 되도록 레퍼런스나 공유 포인터로 다루기 위해, 래핑하여 클래스로 만듦
# cyclegan_cpp_api.cpp
struct ResNetGeneratorImpl : torch::nn::Module {
  torch::nn::Sequential model;
  ResNetGeneratorImpl(int64_t input_nc = 3, int64_t output_nc = 3,
                      int64_t ngf = 64, int64_t n_blocks = 9) {
    TORCH_CHECK(n_blocks >= 0);
    model->push_back(torch::nn::ReflectionPad2d(3)); # 생성자에서 Sequential 컨테이너에 모듈 추가. 이를 통해 for 루프 내 모듈에 대한 변수값 추가
    ...
      model->push_back(torch::nn::Conv2d(
          torch::nn::Conv2dOptions(ngf * mult, ngf * mult * 2, 3)
              .stride(2)
              .padding(1))); # 옵션 사용
    ...
    register_module("model", model);
  }
  Tensor forward(const Tensor &inp) { return model->forward(inp); }
};

TORCH_MODULE(ResNetGenerator); # ResNetGenerator로 ResNetGeneratorImpl를 래핑. 이름이 일치하도록 만드는 것은 중요

In [ ]:
# 파이썬 ResNetGenerator 모델에 대한 완벽한 C++ 대응 구현 완료
# main 함수에서 파라미터 로딩하고 모델 실행
# 이미지를 CImg로 읽고 텐서로 변환하며 텐서를 이미지로 바꾸는 작업은 앞의 작업과 동일
# 추가로 이미지 저장 대신 출력
# cyclegan_cpp_api.cpp
ResNetGenerator model; # 모델 인스턴스화
...
torch::load(model, argv[1]); # 파라미터 로딩
...
cimg_library::CImg<float> image(argv[2]);
image.resize(400, 400);
auto input_ =
    torch::tensor(torch::ArrayRef<float>(image.data(), image.size()));
auto input = input_.reshape({1, 3, image.height(), image.width()});
torch::NoGradGuard no_grad;          # 가드 변수 선언. 기울기 사용 끄는 기간 명시하기 위해 {...} 블럭에 넣는 것도 방법

model->eval();                       # 파이썬처럼 eval 모드 키기

auto output = model->forward(input); # 모델이 아닌 forward 호출
...
cimg_library::CImg<float> out_img(output.data_ptr<float>(),
				    output.size(3), output.size(2),
				    1, output.size(1));
cimg_library::CImgDisplay disp(out_img, "See a C++ API zebra!"); # 이미지를 출력한 후 키 입력 기다릴 필요 있음
while (!disp.is_closed()) {
  disp.wait();
}

In [ ]:
# 흥미로운 변경 사항은 모델을 어떻게 만들고 실행하는지
# 모델 타입으로 변수 선언해 모델을 인스턴스화 함
# torch::load를 통해 모델 로딩(모델 래핑한 점 중요)
# JIT가 저장한 파일로 동작하는 점이 다름

In [ ]:
# 모델 실행 시 with torch.no_grad(): 구문에 해당하는 코드 필요
# 이럴 때는 NoGradGuard 타입 변수 인스턴스화하고 기울기 필요 없는 동안 스코프 안에 살려두면 됨
# 파이썬처럼 model->eval() 호출해 평가 모드로 설정 가능
# 입력 텐서로 model->forward 호출하고 결과도 텐서로 받으며 JIT 개입하지 않으므로 IValue 패킹과 언패킹 필요 없음

## 5절 - 모바일

In [1]:
# 모바일 기기로 모델 배포하는 경우
# 안드로이드로 한정

In [3]:
# 파이토치 C++ 부분인 LibTorch는 안드로이드용으로 컴파일 가능
# 안드로이드 JNI(andriod java native interface)를 사용해 자바로 만든 앱에서 접근 가능
# JIT로 처리된 모델 로딩하고, 입력을 텐서와 IValue로 변환하고, 이를 사용해 모델 실행하고 결과 받는데 필요한 파이토치 함수 필요
# 파이토치 모바일이라 부르는 작은 라이브러리로 래핑되어 있음

In [4]:
# 안드로이드에서 앱 만드는 일반적인 방법은 스튜디오 IDE 사용하는 것
# 안드로이드 스튜디어 템플릿(빈 자바 앱) 중 하나를 사진 찍는 앱으로 변환해서 얼룩말 사이클GAN 모델을 돌리고 결과를 출력할 것임
# 안드로이드 예제 앱 활용

In [ ]:
# 탬플릿 사용하기 위해 필요한 세 가지
# 1. UI 정의
# 사진 찍고 이미지 변환하는 headline이라는 TextView와 사진을 보여주는 ImageView인 image_view 사용
# 사진 찍는 기능은 카메라 앱에 의존

In [ ]:
# 2. 파이토치를 의존성으로 정의: 앱의 build.gradle 파일 편집해 pytorch_andriod와 pytorch_andriod_torchvision 삽입
# build.gradle에 추가할 부분
dependencies {
    ...
    implementation 'org.pytorch:pytorch_android:1.3.0'
    implementation 'org.pytorch:pytorch_android_torchvision:1.3.0'
}

In [ ]:
# 추적한 모델은 assets으로 넣기

In [ ]:
# 3. Activity에서 파생된, 메인 코드를 포함하는 우리의 자바 클래스를 일부 발췌해 살펴보기
# MainActivity.java
...
import org.pytorch.IValue;
import org.pytorch.Module;
import org.pytorch.Tensor;
import org.pytorch.torchvision.TensorImageUtils;
...
public class MainActivity extends AppCompatActivity {
    private org.pytorch.Module model; # JIT로 처리된 모델을 담고 있음

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        ...
        try {
            model = Module.load(assetFilePath(this, "traced_zebra_model.pt")); # 파일에서 모듈 로딩
        } catch (IOException e) {
            Log.e("Zebraify", "Error reading assets", e);
            finish();
        }
        ...
    }
    ...
}

In [5]:
# org.pytorch 네임스페이스에서 임포트할 것이 있음
# 자바이기 때문에 전형적으로 IValue, Module, Tensor 임포트하며, 텐서와 이미지 간의 변한을위해 org.pytorch.torchvision.TensorImageUtils도 사용

In [6]:
# 먼저 모델 담아놓을 변수 선언
# 그리고 액티비티의 onCreate에서 앱 시작되면 Model.load 모세도 사용해 인자로 주어진 위치에서 모듈 읽어들이기
# assets으로 주어지는 앱 데이터는 파일 시스템에서 쉽게 접근하기 어려움
# 따라서 assetFilePath 유틸리티 메소드로 에셋을 파일 시스템 위치에 복사
# 마지막으로 자바에서는 코드가 던지는 모든 예외 잡아줘야 함

In [ ]:
# 안드로이드의 Intent 메커니즘을 사용해 카메라 앱에서 이미지 얻으면 모델에 넣고 돌린 후 사진 표시
# 이 작업은 onActivityResult 이벤트 핸들러가 수행
# MainActivity.java
@Override
protected void onActivityResult(int requestCode, int resultCode, Intent data) {
    if (requestCode == REQUEST_IMAGE_CAPTURE && resultCode == RESULT_OK) { # 카메라 앱이 사진 찍을 때 실행
        // this gets called when the camera app got a picture
        Bitmap bitmap = (Bitmap) data.getExtras().get("data");

        final float[] means = {0.0f, 0.0f, 0.0f}; # 정규화 진행 부분. 기본 이미지가 0...1 범위의 값을 가지므로 변경할 필요가 없음. 따라서 시프트는 0, 스케일은 1로 지정
        final float[] stds = {1.0f, 1.0f, 1.0f};
        // preparing input tensor
        final Tensor inputTensor = TensorImageUtils.bitmapToFloat32Tensor(bitmap, # 비트맵으로 텐서 제작. 토치비전의 ToTensor와 정규화 합친 단계
                means, stds);

        // running the model
        final Tensor outputTensor = model.forward(IValue.from(inputTensor)).toTensor(); # C++과 유사한 부분
        Bitmap output_bitmap = tensorToBitmap(outputTensor, means, stds, Bitmap.Config.RGB_565); # tensorToBitmap은 직접 만든 것임

        ImageView image_view = (ImageView) findViewById(R.id.imageView);
        image_view.setImageBitmap(output_bitmap);
    }
}

In [7]:
# 안드로이드에서 얻은 비트맵을 텐서로 변환하는 작업은 TensorImageUtils.bitmapToFloat32Tensor 함수가 수행
# 두 개의 float 배열인 means와 stds를 bitmap과 함께 지정
# 입력 데이터셋의 평균과 표준편차를 지정해 토치비전의 Normalize 변환처럼 평균 0과 단위 표준편차 가지도록 함
# 안드로이드에서 이미 0..1 범위의 이미지 전송하므로 모델에 바로 넣을 수 있기 때문에 평균은 0, 표준편차는 1로 설정해서 정규화 과정으로 이미지 손상 없게 할 것

In [8]:
# model.forward 호출 부근에서 C++ JIT에서와 같이 IValue 래핑과 언래핑 수행
# 벡터 대신 하나의 IValue를 forward가 입력받는 것이 다름
# 마지막으로 비트맵이 필요한데 파이토치가 이 작업을 하지 않으므로 tensorToBitmap 작성

In [9]:
# 사진 요청하는 코드까지 추가해 만든 지브라파이 앱 화면을 볼 수 있음
# 안드로이드에서 파이토치의 모든 연산에 대한 완전한 버전을 만듦
# 이로 인해 작업에서 사용하지 않는 연산까지 다 포함했으므로 공간 절약을 위해 불필요한 연산 제거할 필요가 있을 거싱ㅁ
# 파이토치 1.4에서는 필요한 연산만을 포함하는 커스텀 파이토치 라이브러리 빌드 가능

#### .1 효율성 개선: 모델 설계와 양자화

In [11]:
# 모바일에서의 모델 동작 연구하다 보면 모델을 더 빨리 동작시키는 것을 생각하게 됨
# 메모리 사용 줄이고 모델 연산을 추적해 내려가면 가장 처음 눈에 띄는 것은 모델 자체 효율화하는 것
# 입출력 사이의 매핑을 동일하지만 더 적은 파라미터와 연산으로 계산하는 방법 -> 증류(distillation)
# 각 가중치 중에 작거나 부적절한 경우 제거하는 경우도 증류에 해당
# 다른 경우로는 신경망의 여러 층을 하나로 합치거나 큰 모델의 출력을 완전히 다르고 단순한 모델로 재현하도록 훈련 시키는 경우도 있음

In [12]:
# 다른 접근 방법으로 각 파라미터와 연산의 크기를 줄이는 것
# 파라미터로 32비트 부동소수점 사용하는 대신 정수로 동작하도록 변환하는 것 -> 양자화(quantization)

In [14]:
# 파이토치는 양자화 텐서를 제공
# torch.float과 torch.double 혹은 torch.long와 비슷한 스칼라 타입으로 제공
# 가장 보편적인 양자화 텐서는 torch.qunit8과 torch.qint8
# 파이토치는 디스패치 메커니즘 사용하기 위해 어런 식의 독립된 스칼라 타입 사용함

In [15]:
# float 대신 정수 사용하면 성능 좋아지지만 약간의 품질 저하 발생
# 품질 저하가 크지 않은 원인으로 두 가지 이유
# 반올림 오차는 근본적으로 무작위로 고려하고 컨볼루션과 선형 계층을 가중 평균으로 간주하면 반올림으로 인한 오차는 어느 정도 상쇄된다고 볼 수 있음
# 이 방식은 float32의 20비트 이상인 상대적인 정밀도를 int가 제공하는 7비트로 줄여줌
# 이 밖에 양자화는 부동소수점을 고정된 정밀도로 바꿔주는 효과 있음
# 즉 제일 큰 값은 7비트 정밀도로 해석되며 제일 큰 값의 1/8 정도의 값은 7-3=4비트를 가짐
# 하지만 L1 정규화가 일어나면 양자화로 인해 작은 값에 대해 적은 정밀도를 가지는 부분을 감당할 수 있을 것으로 생각하며 실제로 많은 경우에 그럼

## 6절 - 최근 기술: 파이토치 모델 엔터프라이즈 서빙

## 7절 - 결론

In [16]:
# 이것으로 모델을 만들어 원하는 곳에 적용하는 방법에 대한 여행 마침
# 제품화된 토치 서빙은 아직 사용하기에는 이른 감이 있지만 잘 만들어지면 JIT로 모델을 내보내기 할 수 있을 거임
# 이제 우리는 모델을 네트워크 서비스나 C++ 애플리케이션, 모바일 등으로 배포하는 법 알게 됨

## 8절 - 연습 문제

In [17]:
# 1. 흥미 유발하는 프로젝트 하나 골라보라. 캐글을 추천한다.